In [1]:
from src.data_preprocessor import DataProcessor
from src.cross_validation import CrossValidation
from src.evaluation import Evaluation
from models.knn import KNN
from models.null_model import NullModelClassification, NullModelRegression
from data_configs.configs import *
import statistics
import numpy as np

config = forest_fires_config
data_processor = DataProcessor(config=config)
cross_validator = CrossValidation(config=config)
classification_nullmodel = NullModelClassification(config=config)
regression_nullmodel = NullModelRegression(config=config)
knn_model = KNN(config)
null_model = NullModelRegression(config=config)

### Data Load and Preprocessing ###

In [2]:
raw_data = data_processor.load_data()

data_1 = data_processor.impute_missing_values(raw_data)

data_2 = data_processor.encode_nominal_features(data_1)

data_3 = data_processor.encode_ordinal_features(data_2)

In [3]:
data_4 = data_processor.log_transform(data_3)

## KNN Model ##

### Hyperparameter Tuning ###

In [4]:
data_train, data_val = cross_validator.random_partition(data_4, random_state=42)

In [5]:
features=['X', 'Y', 'FFMC', 'DMC', 'DC', 'ISI','temp', 'RH', 'wind', 'rain', 'month','day']

In [6]:
gamma = 1/(statistics.stdev(data_train[config['target_column']]))
gamma

0.7265859330636893

In [7]:
statistics.stdev(data_3[config['target_column']])

63.65581846794089

#### Tuning k ####

In [8]:
hyperparameters = np.arange(1,15,1)
scores_dict = {}

for k in hyperparameters: 
    scores = []
    for i, (train_set_1, train_set_2) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):
        
        data_train_standardized = data_processor.standardize_data(train_set_1, train_set_1, features=features)
        data_val_standardized = data_processor.standardize_data(train_set_1,data_val,features=features)  

        predictions_1 = knn_model.knn_regression(data_val_standardized, data_train_standardized, k=k, gamma=gamma)['Predicted Value']
        predictions_1 = data_processor.inverse_log_transform(predictions_1)
        y_true = data_processor.inverse_log_transform(data_val_standardized[config['target_column']])
        score_1 = Evaluation().mean_squared_error(y_true, predictions_1)
        scores.append(score_1)

    average_score = sum(scores) / len(scores)
    print(f"Average MSE score with k={k}: {average_score}")
    scores_dict[k] = average_score

best_k = min(scores_dict, key=scores_dict.get)
print(f"Best k is {best_k} with the lowest average MSE score of {scores_dict[best_k]}")


Average MSE score with k=1: 12827.605644615393
Average MSE score with k=2: 12286.82580512004
Average MSE score with k=3: 12199.416791307332
Average MSE score with k=4: 12164.780424775794
Average MSE score with k=5: 12142.075856759457
Average MSE score with k=6: 12129.468874487815
Average MSE score with k=7: 12124.364856806615
Average MSE score with k=8: 12119.406165249244
Average MSE score with k=9: 12116.522890864095
Average MSE score with k=10: 12112.839825728597
Average MSE score with k=11: 12111.576822898172
Average MSE score with k=12: 12109.65228327208
Average MSE score with k=13: 12108.095866533891
Average MSE score with k=14: 12108.522424369727
Best k is 13 with the lowest average MSE score of 12108.095866533891


#### Tuning Gamma ####

In [9]:
hyperparameters = np.arange(0.4,1.6,0.2)
scores_dict = {}

for g in hyperparameters: 
    scores = []
    for i, (train_set_1, train_set_2) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):
        
        data_train_standardized = data_processor.standardize_data(train_set_1, train_set_1, features=features)
        data_val_standardized = data_processor.standardize_data(train_set_1,data_val,features=features)  

        predictions_1 = knn_model.knn_regression(data_val_standardized, data_train_standardized, k=best_k, gamma=gamma*g)['Predicted Value']
        predictions_1 = data_processor.inverse_log_transform(predictions_1)
        y_true = data_processor.inverse_log_transform(data_val_standardized[config['target_column']])

        score_1 = Evaluation().mean_squared_error(y_true, predictions_1)
        scores.append(score_1)

    average_score = sum(scores) / len(scores)
    print(f"Average MSE score with g={round(g,2)}: {average_score}")
    scores_dict[g] = average_score

best_g = min(scores_dict, key=scores_dict.get)
print(f"Best g is {round(best_g,2)} with the lowest average MSE score of {scores_dict[best_g]}")


Average MSE score with g=0.4: 12091.422644001159
Average MSE score with g=0.6: 12094.075479423424
Average MSE score with g=0.8: 12098.645216298932
Average MSE score with g=1.0: 12108.095866533891
Average MSE score with g=1.2: 12126.838974780072
Average MSE score with g=1.4: 12157.67592372792
Average MSE score with g=1.6: 12198.037354553335
Best g is 0.4 with the lowest average MSE score of 12091.422644001159


### Model Performance ###

In [10]:
mse_scores = []
mae_scores = []
r2_scores = []
pearson_scores = []
null_model_scores = []

# Lists for storing null model metrics
null_model_mae_scores = []
null_model_r2_scores = []
null_model_pearson_scores = []

for i, (train_set, test_set) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):

    data_train_standardized = data_processor.standardize_data(train_set, train_set, features=features)
    data_test_standardized = data_processor.standardize_data(train_set,test_set,features=features)  

    # Train and evaluate 
    predictions_1 = knn_model.knn_regression(data_test_standardized, data_train_standardized, k=best_k, gamma=best_g*gamma)['Predicted Value']
    
    mse_score = Evaluation.mean_squared_error(data_test_standardized[config['target_column']], predictions_1)
    mae_score = Evaluation.mean_absolute_error(data_test_standardized[config['target_column']], predictions_1)
    r2_score = Evaluation.r2_coefficient(data_test_standardized[config['target_column']], predictions_1)
    pearson_score = Evaluation.pearsons_correlation(data_test_standardized[config['target_column']], predictions_1)
    
    mse_scores.append(mse_score)
    mae_scores.append(mae_score)
    r2_scores.append(r2_score)
    pearson_scores.append(pearson_score)

    # Evaluate null model
    null_model_prediction = null_model.naive_regression(test_set)
    null_model_mse = Evaluation.mean_squared_error(test_set[config['target_column']], null_model_prediction)
    null_model_mae = Evaluation.mean_absolute_error(test_set[config['target_column']], null_model_prediction)
    null_model_r2 = Evaluation.r2_coefficient(test_set[config['target_column']], null_model_prediction)
    null_model_pearson = Evaluation.pearsons_correlation(test_set[config['target_column']], null_model_prediction)
    
    null_model_scores.append(null_model_mse)
    null_model_mae_scores.append(null_model_mae)
    null_model_r2_scores.append(null_model_r2)
    null_model_pearson_scores.append(null_model_pearson)


average_mse_score = sum(mse_scores) / len(mse_scores)
average_mae_score = sum(mae_scores) / len(mae_scores)
average_r2_score = sum(r2_scores) / len(r2_scores)
average_pearson_score = sum(pearson_scores) / len(pearson_scores)
average_null_model_mse = sum(null_model_scores) / len(null_model_scores)
average_null_model_mae = sum(null_model_mae_scores) / len(null_model_mae_scores)
average_null_model_r2 = sum(null_model_r2_scores) / len(null_model_r2_scores)
average_null_model_pearson = sum(null_model_pearson_scores) / len(null_model_pearson_scores)

print(f"Average null model MSE score: {average_null_model_mse}")
print(f"Average null model MAE score: {average_null_model_mae}")
print(f"Average null model r2 score: {average_null_model_r2}")
print(f"Average null model pearson score: {average_null_model_pearson}")
print(f"Average MSE score for k={best_k}, g={round(best_g,2)}: {average_mse_score}")
print(f"Average MAE score for k={best_k}, g={round(best_g,2)}: {average_mae_score}")
print(f"Average r2 score for k={best_k}, g={round(best_g,2)}: {average_r2_score}")
print(f"Average pearson score for k={best_k}, g={round(best_g,2)}: {average_pearson_score}")


c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning

Average null model MSE score: 1.886024856693518
Average null model MAE score: 1.143350282297318
Average null model r2 score: 0.0
Average null model pearson score: nan
Average MSE score for k=13, g=0.4: 2.0404650445322154
Average MAE score for k=13, g=0.4: 1.1560737375015546
Average r2 score for k=13, g=0.4: -0.08313266642576791
Average pearson score for k=13, g=0.4: 0.04985113191149566


c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


## Edited KNN ##

### Hyperparameter Tuning ###

In [11]:
epsilon = statistics.stdev(data_4[config['target_column']])
epsilon

1.398435955883445

#### Tuning Epsilon ####

In [12]:
hyperparameters = np.arange(0.1,2,0.2)
scores_dict = {}

for e in hyperparameters: 
    scores = []
    for i, (train_set_1, train_set_2) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):
        
        edited_train_set = knn_model.edited_knn_regression(train_set_1,epsilon=epsilon*e)

        data_train_standardized = data_processor.standardize_data(edited_train_set, edited_train_set, features=features)
        data_val_standardized = data_processor.standardize_data(edited_train_set,data_val,features=features)  

        predictions_1 = knn_model.knn_regression(data_val_standardized, data_train_standardized, k=best_k, gamma=gamma*best_g)['Predicted Value']
        predictions_1 = data_processor.inverse_log_transform(predictions_1)
        y_true = data_processor.inverse_log_transform(data_val_standardized[config['target_column']])
        score_1 = Evaluation().mean_squared_error(y_true, predictions_1)
        scores.append(score_1)

    average_score = sum(scores) / len(scores)
    print(f"Average MSE score with e={round(e,2)}: {average_score}")
    scores_dict[e] = average_score

best_e = min(scores_dict, key=scores_dict.get)
print(f"Best e is {round(best_e,2)} with the lowest average MSE score of {scores_dict[best_e]}")


Average MSE score with e=0.1: 12165.574650887835
Average MSE score with e=0.3: 12158.017086867718
Average MSE score with e=0.5: 12147.369137096961
Average MSE score with e=0.7: 12142.41722097133
Average MSE score with e=0.9: 12135.54869684796
Average MSE score with e=1.1: 12131.178096475358
Average MSE score with e=1.3: 12128.983187748141
Average MSE score with e=1.5: 12124.998699477748
Average MSE score with e=1.7: 12123.840198648306
Average MSE score with e=1.9: 12123.837560847485
Best e is 1.9 with the lowest average MSE score of 12123.837560847485


#### Tuning k ####

In [13]:
hyperparameters = np.arange(1,10,1)
scores_dict = {}

for k in hyperparameters: 
    scores = []
    for i, (train_set_1, train_set_2) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):
        
        edited_train_set = knn_model.edited_knn_regression(train_set_1,epsilon=epsilon*best_e)

        data_train_standardized = data_processor.standardize_data(edited_train_set, edited_train_set, features=features)
        data_val_standardized = data_processor.standardize_data(edited_train_set,data_val,features=features)  

        predictions_1 = knn_model.knn_regression(data_val_standardized, data_train_standardized, k=k, gamma=gamma*best_g)['Predicted Value']
        predictions_1 = data_processor.inverse_log_transform(predictions_1)
        y_true = data_processor.inverse_log_transform(data_val_standardized[config['target_column']])
        score_1 = Evaluation().mean_squared_error(y_true, predictions_1)
        scores.append(score_1)

    average_score = sum(scores) / len(scores)
    print(f"Average MSE score with k={k}: {average_score}")
    scores_dict[k] = average_score

best_k = min(scores_dict, key=scores_dict.get)
print(f"Best k is {best_k} with the lowest average MSE score of {scores_dict[best_k]}")


Average MSE score with k=1: 12207.134292500006
Average MSE score with k=2: 12154.633212721372
Average MSE score with k=3: 12140.775267079347
Average MSE score with k=4: 12136.203670137307
Average MSE score with k=5: 12133.043048935131
Average MSE score with k=6: 12131.608968657854
Average MSE score with k=7: 12131.666063553013
Average MSE score with k=8: 12130.92504046272
Average MSE score with k=9: 12129.313225891732
Best k is 9 with the lowest average MSE score of 12129.313225891732


#### Tuning gamma ####

In [14]:
hyperparameters = np.arange(0.4,1.6,0.2)
scores_dict = {}

for g in hyperparameters: 
    scores = []
    for i, (train_set_1, train_set_2) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):
        
        edited_train_set = knn_model.edited_knn_regression(train_set_1,epsilon=epsilon*best_e)

        data_train_standardized = data_processor.standardize_data(edited_train_set, edited_train_set, features=features)
        data_val_standardized = data_processor.standardize_data(edited_train_set,data_val,features=features)  

        predictions_1 = knn_model.knn_regression(data_val_standardized, data_train_standardized, k=best_k, gamma=g*gamma)['Predicted Value']
        predictions_1 = data_processor.inverse_log_transform(predictions_1)
        y_true = data_processor.inverse_log_transform(data_val_standardized[config['target_column']])
        score_1 = Evaluation().mean_squared_error(y_true, predictions_1)
        scores.append(score_1)

    average_score = sum(scores) / len(scores)
    print(f"Average MSE score with g={round(g,2)}: {average_score}")
    scores_dict[g] = average_score

best_g = min(scores_dict, key=scores_dict.get)
print(f"Best g is {round(best_g,2)} with the lowest average MSE score of {scores_dict[best_g]}")


Average MSE score with g=0.4: 12129.313225891732
Average MSE score with g=0.6: 12129.954313819688
Average MSE score with g=0.8: 12131.251570720604
Average MSE score with g=1.0: 12133.293406358649
Average MSE score with g=1.2: 12135.859797307752
Average MSE score with g=1.4: 12138.616298499108
Average MSE score with g=1.6: 12141.34307683484
Best g is 0.4 with the lowest average MSE score of 12129.313225891732


In [15]:
mse_scores = []
mae_scores = []
r2_scores = []
pearson_scores = []
null_model_scores = []

# Lists to store null model metrics for MAE, R^2, and Pearson
null_model_mae_scores = []
null_model_r2_scores = []
null_model_pearson_scores = []

for i, (train_set, test_set) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):

    edited_train_set = knn_model.edited_knn_regression(train_set,epsilon=epsilon*best_e)

    data_train_standardized = data_processor.standardize_data(edited_train_set, edited_train_set, features=features)
    data_test_standardized = data_processor.standardize_data(edited_train_set,test_set,features=features)  

    predictions_1 = knn_model.knn_regression(data_test_standardized, data_train_standardized, k=best_k, gamma=best_g*gamma)['Predicted Value']

    mse_score = Evaluation.mean_squared_error(data_test_standardized[config['target_column']], predictions_1)
    mae_score = Evaluation.mean_absolute_error(data_test_standardized[config['target_column']], predictions_1)
    r2_score = Evaluation.r2_coefficient(data_test_standardized[config['target_column']], predictions_1)
    pearson_score = Evaluation.pearsons_correlation(data_test_standardized[config['target_column']], predictions_1)

    mse_scores.append(mse_score)
    mae_scores.append(mae_score)
    r2_scores.append(r2_score)
    pearson_scores.append(pearson_score)
    
    # Null model evaluation
    null_model_prediction = null_model.naive_regression(test_set)
    null_model_mse = Evaluation.mean_squared_error(test_set[config['target_column']], null_model_prediction)
    null_model_mae = Evaluation.mean_absolute_error(test_set[config['target_column']], null_model_prediction)
    null_model_r2 = Evaluation.r2_coefficient(test_set[config['target_column']], null_model_prediction)
    null_model_pearson = Evaluation.pearsons_correlation(test_set[config['target_column']], null_model_prediction)

    null_model_scores.append(null_model_mse)
    null_model_mae_scores.append(null_model_mae)
    null_model_r2_scores.append(null_model_r2)
    null_model_pearson_scores.append(null_model_pearson)

average_mse_score = sum(mse_scores) / len(mse_scores)
average_mae_score = sum(mae_scores) / len(mae_scores)
average_r2_score = sum(r2_scores) / len(r2_scores)
average_pearson_score = sum(pearson_scores) / len(pearson_scores)
average_null_model_mse = sum(null_model_scores) / len(null_model_scores)
average_null_model_mae = sum(null_model_mae_scores) / len(null_model_mae_scores)
average_null_model_r2 = sum(null_model_r2_scores) / len(null_model_r2_scores)
average_null_model_pearson = sum(null_model_pearson_scores) / len(null_model_pearson_scores)

print(f"Average null model MSE score: {average_null_model_mse}")
print(f"Average null model MAE score: {average_null_model_mae}")
print(f"Average null model R2 score: {average_null_model_r2}")
print(f"Average null model Pearson score: {average_null_model_pearson}")
print(f"Average MSE score for k={best_k}, g={round(best_g,2)}: {average_mse_score}")
print(f"Average MAE score for k={best_k}, g={round(best_g,2)}: {average_mae_score}")
print(f"Average R2 score for k={best_k}, g={round(best_g,2)}: {average_r2_score}")
print(f"Average Pearson score for k={best_k}, g={round(best_g,2)}: {average_pearson_score}")


c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning

Average null model MSE score: 1.886024856693518
Average null model MAE score: 1.143350282297318
Average null model R2 score: 0.0
Average null model Pearson score: nan
Average MSE score for k=9, g=0.4: 2.12425386194167
Average MAE score for k=9, g=0.4: 1.1266111989002865
Average R2 score for k=9, g=0.4: -0.1274425876833161
Average Pearson score for k=9, g=0.4: 0.019005132239857864


c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


## Condensed Knn ##

#### Tuning Epsilon ####

In [16]:
hyperparameters = np.arange(0.1,1.5,0.1)
scores_dict = {}

for e in hyperparameters: 
    scores = []
    for i, (train_set_1, train_set_2) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):
        
        condesed_train_set = knn_model.condensed_knn_regression(train_set_1,epsilon=epsilon*e)
        data_train_standardized = data_processor.standardize_data(condesed_train_set, condesed_train_set, features=features)
        data_val_standardized = data_processor.standardize_data(condesed_train_set,data_val,features=features)

        predictions_1 = knn_model.knn_regression(data_val_standardized, data_train_standardized, k=best_k, gamma=gamma*best_g)['Predicted Value']
        predictions_1 = data_processor.inverse_log_transform(predictions_1)
        y_true = data_processor.inverse_log_transform(data_val_standardized[config['target_column']])
        score_1 = Evaluation().mean_squared_error(y_true, predictions_1)
        scores.append(score_1)

    average_score = sum(scores) / len(scores)
    print(f"Average MSE score with e={round(e,2)}: {average_score}")
    scores_dict[e] = average_score

best_e = min(scores_dict, key=scores_dict.get)
print(f"Best e is {round(best_e,2)} with the lowest average MSE score of {scores_dict[best_e]}")


Average MSE score with e=0.1: 12075.259611584006
Average MSE score with e=0.2: 12076.654273473125
Average MSE score with e=0.3: 12079.006592309124
Average MSE score with e=0.4: 12077.504025003353
Average MSE score with e=0.5: 12072.026349222988
Average MSE score with e=0.6: 12076.267932055027
Average MSE score with e=0.7: 12067.40138643712
Average MSE score with e=0.8: 12061.690153070402
Average MSE score with e=0.9: 12069.853343368715
Average MSE score with e=1.0: 12071.606768185511
Average MSE score with e=1.1: 12071.53817625077
Average MSE score with e=1.2: 12063.935247966929
Average MSE score with e=1.3: 12060.465054910735
Average MSE score with e=1.4: 12055.323152810193
Best e is 1.4 with the lowest average MSE score of 12055.323152810193


#### Tuning k ####

In [17]:
hyperparameters = np.arange(1,10,1)
scores_dict = {}

for k in hyperparameters: 
    scores = []
    for i, (train_set_1, train_set_2) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):
        
        condesed_train_set = knn_model.condensed_knn_regression(train_set_1,epsilon=epsilon*best_e)

        data_train_standardized = data_processor.standardize_data(condesed_train_set, condesed_train_set, features=features)
        data_val_standardized = data_processor.standardize_data(condesed_train_set,data_val,features=features)

        predictions_1 = knn_model.knn_regression(data_val_standardized, data_train_standardized, k=k, gamma=gamma*best_g)['Predicted Value']
        predictions_1 = data_processor.inverse_log_transform(predictions_1)
        y_true = data_processor.inverse_log_transform(data_val_standardized[config['target_column']])
        score_1 = Evaluation().mean_squared_error(y_true, predictions_1)
        scores.append(score_1)

    average_score = sum(scores) / len(scores)
    print(f"Average MSE score with k={k}: {average_score}")
    scores_dict[k] = average_score

best_k = min(scores_dict, key=scores_dict.get)
print(f"Best k is {best_k} with the lowest average MSE score of {scores_dict[best_k]}")


Average MSE score with k=1: 13821.64086932693
Average MSE score with k=2: 12446.119128893692
Average MSE score with k=3: 12151.488963346674
Average MSE score with k=4: 12068.526894746252
Average MSE score with k=5: 12066.530979052917
Average MSE score with k=6: 12053.613244974931
Average MSE score with k=7: 12065.413830988477
Average MSE score with k=8: 12045.771812894218
Average MSE score with k=9: 12060.497023792854
Best k is 8 with the lowest average MSE score of 12045.771812894218


#### Tuning Gamma ####

In [18]:
hyperparameters = np.arange(0.4,1.6,0.2)
scores_dict = {}

for g in hyperparameters: 
    scores = []
    for i, (train_set_1, train_set_2) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):
        
        condesed_train_set = knn_model.condensed_knn_regression(train_set_1,epsilon=epsilon*best_e)

        data_train_standardized = data_processor.standardize_data(condesed_train_set, condesed_train_set, features=features)
        data_val_standardized = data_processor.standardize_data(condesed_train_set,data_val,features=features)  

        predictions_1 = knn_model.knn_regression(data_val_standardized, data_train_standardized, k=best_k, gamma=g*gamma)['Predicted Value']
        predictions_1 = data_processor.inverse_log_transform(predictions_1)
        
        y_true = data_processor.inverse_log_transform(data_val_standardized[config['target_column']])
        score_1 = Evaluation().mean_squared_error(y_true, predictions_1)
        scores.append(score_1)

    average_score = sum(scores) / len(scores)
    print(f"Average MSE score with g={round(g,2)}: {average_score}")
    scores_dict[g] = average_score

best_g = min(scores_dict, key=scores_dict.get)
print(f"Best g is {round(best_g,2)} with the lowest average MSE score of {scores_dict[best_g]}")


Average MSE score with g=0.4: 12051.15301905379
Average MSE score with g=0.6: 12071.13181252332
Average MSE score with g=0.8: 12087.37371571708
Average MSE score with g=1.0: 12130.083804803991
Average MSE score with g=1.2: 12205.976299441445
Average MSE score with g=1.4: 12237.392410435472
Average MSE score with g=1.6: 12285.221735620984
Best g is 0.4 with the lowest average MSE score of 12051.15301905379


### Model Performance ###

In [19]:
mse_scores = []
mae_scores = []
r2_scores = []
pearson_scores = []
null_model_scores = []

# Lists to store null model metrics for MAE, R^2, and Pearson
null_model_mae_scores = []
null_model_r2_scores = []
null_model_pearson_scores = []

for i, (train_set, test_set) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):

    condesed_train_set = knn_model.condensed_knn_regression(train_set,epsilon=epsilon*best_e)

    data_train_standardized = data_processor.standardize_data(condesed_train_set, condesed_train_set, features=features)
    data_test_standardized = data_processor.standardize_data(edited_train_set,test_set,features=features)  

    predictions_1 = knn_model.knn_regression(data_test_standardized, data_train_standardized, k=best_k, gamma=best_g*gamma)['Predicted Value']

    mse_score = Evaluation.mean_squared_error(data_test_standardized[config['target_column']], predictions_1)
    mae_score = Evaluation.mean_absolute_error(data_test_standardized[config['target_column']], predictions_1)
    r2_score = Evaluation.r2_coefficient(data_test_standardized[config['target_column']], predictions_1)
    pearson_score = Evaluation.pearsons_correlation(data_test_standardized[config['target_column']], predictions_1)

    mse_scores.append(mse_score)
    mae_scores.append(mae_score)
    r2_scores.append(r2_score)
    pearson_scores.append(pearson_score)
    
    # Null model evaluation
    null_model_prediction = null_model.naive_regression(test_set)
    null_model_mse = Evaluation.mean_squared_error(test_set[config['target_column']], null_model_prediction)
    null_model_mae = Evaluation.mean_absolute_error(test_set[config['target_column']], null_model_prediction)
    null_model_r2 = Evaluation.r2_coefficient(test_set[config['target_column']], null_model_prediction)
    null_model_pearson = Evaluation.pearsons_correlation(test_set[config['target_column']], null_model_prediction)

    null_model_scores.append(null_model_mse)
    null_model_mae_scores.append(null_model_mae)
    null_model_r2_scores.append(null_model_r2)
    null_model_pearson_scores.append(null_model_pearson)


average_mse_score = sum(mse_scores) / len(mse_scores)
average_mae_score = sum(mae_scores) / len(mae_scores)
average_r2_score = sum(r2_scores) / len(r2_scores)
average_pearson_score = sum(pearson_scores) / len(pearson_scores)
average_null_model_mse = sum(null_model_scores) / len(null_model_scores)
average_null_model_mae = sum(null_model_mae_scores) / len(null_model_mae_scores)
average_null_model_r2 = sum(null_model_r2_scores) / len(null_model_r2_scores)
average_null_model_pearson = sum(null_model_pearson_scores) / len(null_model_pearson_scores)


print(f"Average null model MSE score: {average_null_model_mse}")
print(f"Average null model MAE score: {average_null_model_mae}")
print(f"Average null model R2 score: {average_null_model_r2}")
print(f"Average null model Pearson score: {average_null_model_pearson}")
print(f"Average MSE score for k={best_k}, g={round(best_g,2)}: {average_mse_score}")
print(f"Average MAE score for k={best_k}, g={round(best_g,2)}: {average_mae_score}")
print(f"Average R2 score for k={best_k}, g={round(best_g,2)}: {average_r2_score}")
print(f"Average Pearson score for k={best_k}, g={round(best_g,2)}: {average_pearson_score}")


c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning

Average null model MSE score: 1.886024856693518
Average null model MAE score: 1.143350282297318
Average null model R2 score: 0.0
Average null model Pearson score: nan
Average MSE score for k=8, g=0.4: 2.3195308891008524
Average MAE score for k=8, g=0.4: 1.2739057598614554
Average R2 score for k=8, g=0.4: -0.23697736027759547
Average Pearson score for k=8, g=0.4: 0.019951302249793947


c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
